In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<font color = "#CC3D3D"><p>
# Topics
* [Group Aggregation](#Group-Aggregation)
* [Pivot Tables](#Pivot-Tables)
* [Merging DataFrames](#Merging-DataFrames)
* [Appending DataFrames](#Appending-DataFrames)

## Group Aggregation
<br><img align="left" src="http://drive.google.com/uc?export=view&id=17lLj-fLLYk6Dxcz7yBIX7bMEl4PAESBB" width=800 height=600>

In [2]:
df = DataFrame({'key'  : ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                'data' : [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

,key,data
0,A,0
1,B,5
2,C,10
3,A,5
4,B,10
5,C,15
6,A,10
7,B,15
8,C,20


<font color = 'blue'>Typical usage

In [3]:
df.groupby('key')['data'].sum() #key로 그룹화하여 'data'의 값을 모두 더함.

key
A    15
B    30
C    45
Name: data, dtype: int64

In [12]:
type(df.groupby('key')['data'].sum()) # groupby의 결과는 Series

pandas.core.series.Series

In [7]:
df.groupby('key')['data'] #.sum()과 같이 값을 연산하는 것이 없다면 데이터 위치만을 반환.

In [9]:
df.groupby('key')['data'].size()  # size()는 결측값 포함해서 빈도 측정, count()는 결측값 제외 

key
A    3
B    3
C    3
Name: data, dtype: int64

In [10]:
df.key.value_counts() #key 칼럼에 있는 값들의 개수를 반환.

A    3
C    3
B    3
Name: key, dtype: int64

In [5]:
# When we reset the index, the old index is added as a column, and a new sequential index is used
df.groupby('key')['data'].sum().reset_index() #reset_index를 통해 데이터를 확인하기 편함.

,key,data
0,A,15
1,B,30
2,C,45


In [6]:
df.groupby('key', as_index=False)['data'].sum()

,key,data
0,A,15
1,B,30
2,C,45


In [14]:
type(df.groupby('key', as_index=False)['data'].sum())
#as_index= false를 지정하면 dataframe 형식으로 반환됨.

pandas.core.frame.DataFrame

<font color = 'blue'>Iterating through groups

In [23]:
grouped = df.groupby('key')['data']
for name, group in grouped:
    print(name)
    print(group)

A
0     0
3     5
6    10
Name: data, dtype: int64
B
1     5
4    10
7    15
Name: data, dtype: int64
C
2    10
5    15
8    20
Name: data, dtype: int64


In [29]:
grouped = df.groupby('key')['data']
for a, b in grouped:
    print(a)
    print("_____________")
    print(b)
    print("_____________")
    #print(c)
    

A
_____________
0     0
3     5
6    10
Name: data, dtype: int64
_____________
B
_____________
1     5
4    10
7    15
Name: data, dtype: int64
_____________
C
_____________
2    10
5    15
8    20
Name: data, dtype: int64
_____________


In [17]:
type(grouped)

pandas.core.groupby.generic.SeriesGroupBy

In [8]:
[v for v in grouped.get_group('A')]
#grouped는 key에 대해서 data를 반환하는 값.

[0, 5, 10]

<font color = 'blue'>Filtration

In [30]:
print(df) #데이터 프레임을 print
df.groupby('key')['data'].sum() #시리즈형태로 반환됨.

  key  data
0   A     0
1   B     5
2   C    10
3   A     5
4   B    10
5   C    15
6   A    10
7   B    15
8   C    20


key
A    15
B    30
C    45
Name: data, dtype: int64

In [35]:
df.groupby('key')['data'].filter(lambda x: x.sum() > 30).reset_index()
#filter : groupby해서 합이 30이상인 행. 즉 key가 C에 해당하는 행을 인덱스와 함께 보여줌.

,index,data
0,2,10
1,5,15
2,8,20


In [3]:
df

,key,data
0,A,0
1,B,5
2,C,10
3,A,5
4,B,10
5,C,15
6,A,10
7,B,15
8,C,20


<font color = 'blue'>Using agg method

In [4]:
df.groupby('key')['data'].agg('max')
#.agg(['max', 'min']) 여러개의 집계함수를 사용하고 싶거나 자신이 만든 함수를 사용하고 싶을 때 agg사용

key
A    10
B    15
C    20
Name: data, dtype: int64

In [37]:
df.groupby('key')['data'].max()

key
A    10
B    15
C    20
Name: data, dtype: int64

In [39]:
df.groupby('key')['data'].min()

key
A     0
B     5
C    10
Name: data, dtype: int64

In [5]:
df.groupby('key')['data'].agg(lambda x: x.max() - x.min())
#agg(집계함수)에서는 두 함수에 대한 연산이 가능함.

key
A    10
B    10
C    10
Name: data, dtype: int64

In [49]:
df.groupby('key')['data'].agg(lambda x: x.max() - x.min()).reset_index()

,key,data
0,A,10
1,B,10
2,C,10


In [6]:
df.groupby('key')['data'].filter(lambda x: x.max() - x.min())
#filter는 조건에 해당하는 행을 가져오기 위함.

0     0
1     5
2    10
3     5
4    10
5    15
6    10
7    15
8    20
Name: data, dtype: int64

<font color = 'blue'>Applying multiple functions at once

In [53]:
df.groupby('key')['data'].agg(['mean', 'std']).reset_index() #평균과 표준편차

,key,mean,std
0,A,5,5.0
1,B,10,5.0
2,C,15,5.0


<font color = 'blue'>Named aggregation

In [14]:
df.groupby('key')['data'].agg([('평균','mean'), ('표준편차','std')]).reset_index()
#column의 이름을 변경하고 싶을 때 ()튜플과 []리스트을 사용함.

,key,평균,표준편차
0,A,5,5.0
1,B,10,5.0
2,C,15,5.0


<font color = 'blue'>By default, all of the numeric columns are aggregated.

In [58]:
df['average'] = df.data.apply(lambda x, y: 'above' if x > y else 'below', args=(df.data.mean(),))
df

#lambda부분: x = df.data, y = df.data.mean()(data 칼럼 모든 값의 평균값.)
#df.data의 값이 전체 평균보다 크다면  --> x>y 이면: 'above' 아니면 'below'

,key,data,average
0,A,0,below
1,B,5,below
2,C,10,below
3,A,5,below
4,B,10,below
5,C,15,above
6,A,10,below
7,B,15,above
8,C,20,above


In [9]:
df['average2'] = df.data.apply(lambda x, x.mean(): 'above' if x > x.mean() else 'below')

SyntaxError: invalid syntax (<ipython-input-9-3577a9c88d30>, line 1)

In [57]:
df.data.mean()

10.0

In [16]:
df.groupby(['key']).mean() #만약 칼럼을 생략하면 숫자 형식의 칼럼을 해당 함수로 연산해줌.

,data
key,
A,5
B,10
C,15


In [62]:
df.groupby('key').mean() #dataframe 형식

,data
key,
A,5
B,10
C,15


In [63]:
type(df.groupby('key').mean())

pandas.core.frame.DataFrame

<font color = 'blue'>Applying different functions to DataFrame columns

In [64]:
df.groupby('key').agg({'data' : [('평균','mean'), ('합계', np.sum)],
                       'average' : [('범주',Series.nunique)]}).reset_index()

key data     average
        평균  합계      범주
0   A    5  15       1
1   B   10  30       2
2   C   15  45       2

## Pivot Tables
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1HEBp4qq4GaksdQBb2fx2tRsq-gVR-j6d" width=800 height=600>

In [65]:
tr = pd.DataFrame({
        'id': [1,1,1,1,2,2,2],
        'site': ['a','b','c','a','a','b','b'],
        'pageview': np.arange(1,8),
        'dwelltime': np.arange(7.0, 0, -1),
    }, columns=['id','site','pageview','dwelltime'])

tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


In [66]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum)

site,a,b,c
id,,,
1,5.0,2.0,3.0
2,5.0,13.0,NaN


In [69]:
tr.groupby(['id', 'site'])['pageview'].sum().reset_index()
#groupby를 통해서도 2개 이상의 groupby 결과를 볼수 있음. 그러나 null값은 반환하지 않음.
#pd.pivot_table()을 사용한다면 두개의 groupby 결과를 시각적으로 볼수 있음.

,id,site,pageview
0,1,a,5
1,1,b,2
2,1,c,3
3,2,a,5
4,2,b,13


In [72]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0)

#fill_value()를 통해서 결측값(NAN)에 0을 채움.

site,a,b,c
id,,,
1,5,2,3
2,5,13,0


In [74]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0).reset_index()

#reset_index()

site,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [75]:
pv = pd.pivot_table(tr, values='pageview', index='id', columns='site',
                    aggfunc=sum, fill_value=0).reset_index()
pv.columns.name = None
pv
#칼럼 이름을 삭제.

,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [23]:
pd.pivot_table(tr, values='pageview', index='id', 
               aggfunc=np.mean, fill_value=0).reset_index()

,id,pageview
0,1,2.5
1,2,6.0


In [80]:
tr.groupby('id')['pageview'].agg(np.mean).reset_index()
# 피벗테이블의 index에 한개의 인자를 넣는다면 한개의 칼럼으로 groupby하는 것과 같음.

,id,pageview
0,1,2.5
1,2,6.0


In [81]:
pd.pivot_table(tr, values='pageview', columns='site', 
               aggfunc=np.size, fill_value=0)

site,a,b,c
pageview,3,3,1


In [82]:
tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


## Merging DataFrames

In [85]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [86]:
display(df1)
display(df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [87]:
pd.merge(df1, df2)
#merge할 경우 df1과 df2에 공통으로 존재하는 key 칼럼을 통해서 merge함.

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If not specified, merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [29]:
pd.merge(df1, df2, on='key') #on 파라미터를 통해서 어떤 칼럼을 기준으로 merge할지를 정함.

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If the column names are different in each object, you can specify them separately:

In [88]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [93]:
display(df3)
display(df4)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [89]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')
#lkey와 rkey를 기준으로 merge (칼럼의 이름이 다르다면 직접 입력해야함.)

#결과 값을 보면 key에 공통으로 존재하지 않는다면 삭제되는 행이 존재함.

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [94]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how = "inner")
#merge 함수의 how 매개변수에 대한 기본값은 inner이다.

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


<font color='darkgreen'><p>
##### Merge Types
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1yEU_xZ9qZrZi8IIWTMhhMIKI95XxisX0" width=900 height=800>

# left에 있는 값의 결측값 존재를 부정한다면 left join을 해야함.

<font color = 'blue'>By default merge does an 'inner' join; the keys in the result are the intersection. The outer join takes the union of the keys:

In [32]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


<font color = 'blue'>Pandas also provides the *merge* DataFrame method:

In [33]:
df1.merge(df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


## Appending DataFrames
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1lSpzDfbRlY_mTlJH0t1xZADJUvbHyPMw" width=800 height=600>

In [95]:
df5 = DataFrame({'key': ['a', 'b', 'd'], 'data1': range(3)})
print('df1:\n', df1)
print('df5:\n',df5)

df1:
   key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
df5:
   key  data1
0   a      0
1   b      1
2   d      2


In [96]:
pd.concat([df1, df5])

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
0,a,0
1,b,1
2,d,2


In [97]:
pd.concat([df1, df5], axis=1)

,key,data1,key,data1
0,b,0,a,0.0
1,b,1,b,1.0
2,a,2,d,2.0
3,c,3,NaN,NaN
4,a,4,NaN,NaN
5,a,5,NaN,NaN
6,b,6,NaN,NaN


<font color = "blue"><p>
## Exercises

In [10]:
cs = pd.read_csv('Demo.csv', encoding='cp949', engine='python')
tr = pd.read_csv('구매내역정보.csv', encoding='cp949', engine='python')

In [124]:
cs.head()

,ID,성별,연령,거주지역
0,478207946,1,84,서울 성동구
1,479806984,1,84,서울 서초구
2,94790213,1,84,부산 사상구
3,656026338,1,84,서울 영등포구
4,433076833,1,83,경기 고양시


In [125]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0


In [126]:
#연습 -
tr.groupby('ID').agg({
    '상품중분류명': [('구매다양성',Series.nunique)],
    '구매일자' :[('최초구매일', np.min)],
    '구매금액': [('총구매액',np.sum)]
}).reset_index()

,ID,상품중분류명,구매일자,구매금액
,,구매다양성,최초구매일,총구매액
0,741463,3,20140115,560000
1,909249,12,20140118,640000
2,1065068,7,20140706,1103000
3,1310410,2,20140119,249000
4,1387032,3,20140118,113000
...,...,...,...,...
3537,998623524,11,20140221,2088000
3538,999024423,3,20140517,25000
3539,999184958,11,20140115,3147000


In [16]:
tr.groupby('ID').agg({
    '상품중분류명': [('구매다양성', lambda x: x.nunique())],
    '구매일자' :[('최초구매일', np.min)],
    '구매금액': [('총구매액',np.sum)]
}).reset_index()

,ID,상품중분류명,구매일자,구매금액
,,구매다양성,최초구매일,총구매액
0,741463,3,20140115,560000
1,909249,12,20140118,640000
2,1065068,7,20140706,1103000
3,1310410,2,20140119,249000
4,1387032,3,20140118,113000
...,...,...,...,...
3537,998623524,11,20140221,2088000
3538,999024423,3,20140517,25000
3539,999184958,11,20140115,3147000


**[연습문제 34]** `60대 여성` 고객리스트를 출력하시오.

In [127]:
cs.query('성별 == 1 and 60 <= 연령 <= 69')

,ID,성별,연령,거주지역
224,868529581,1,69,서울 송파구
227,663826861,1,69,서울 서대문구
230,410362886,1,69,부산 영도구
231,904352987,1,69,서울 동대문구
240,696566991,1,69,서울 노원구
...,...,...,...,...
3522,657285731,1,61,경기 수원시
3523,257825183,1,61,부산 남구
3533,765568307,1,69,부산 수영구
3536,164097706,1,67,경기 안양시


**[연습문제 35]** `남성고객`과 `여성고객`은 각각 몇명인가?

In [128]:
cs.groupby('성별')['ID'].size().reset_index() 

,성별,ID
0,1,859
1,2,2683


In [163]:
pd.pivot_table(cs, values = 'ID', index = '성별',
               aggfunc = np.size)
#np안의 size를 사용.

,ID
성별,
1,859
2,2683


여자("1")가 859 <p/>
남자("2")가 2683

**[연습문제 36]** `여성고객`의 `평균나이`는 얼마인가?

In [130]:
cs.query('성별 == 1').연령.mean()

62.442374854481955

In [133]:
cs.query('성별 == 1')['연령'].mean()

62.442374854481955

**[연습문제 37]** `70대 여성 고객`들은 주로 어느 `지역`에 거주하고 있는가? 상위 5개 지역만 나열하시오.

In [136]:
cs.query('성별 == 1 and 70 <= 연령 <= 79').거주지역.value_counts().index[:5].to_list

<bound method IndexOpsMixin.tolist of Index(['경기 안양시', '서울 동대문구', '서울 강남구', '경기 성남시', '서울 송파구'], dtype='object')>

In [135]:
cs.query('성별 == 1 and 70 <= 연령 <= 79').거주지역.value_counts().head(5) #value_counts()를 사용하면 내림차순으로 정렬됨.

경기 안양시     10
서울 동대문구     9
서울 강남구      7
경기 성남시      6
서울 송파구      6
Name: 거주지역, dtype: int64

In [140]:
type(cs.query('성별 == 1 and 70 <= 연령 <= 79'))

pandas.core.frame.DataFrame

In [156]:
type(cs.query('성별 == 1 and 70 <= 연령 <= 79').거주지역)
#query를 사용할 경우에는 dataframe
#query를 사용후 인덱싱할 경우 Series 타입이됨. 따라서 value_counts()를 사용할 수 있음.

pandas.core.series.Series

In [157]:
cs.query('성별 == 1 and 70 <= 연령 <= 79').shape[0]

131

In [158]:
len(cs.query('성별 == 1 and 70 <= 연령 <= 79'))

131

**[연습문제 38]** (상품중분류명 기준) `Best seller`는 무엇인가?

In [164]:
tr.상품중분류명.value_counts().index[0]
#상품중분류명의 값을 count해서 가장 많이 팔리는 순서로 정렬. 이후 index로 해당 하는 상품중분류명 추출

'가공식품'

**[연습문제 39]** `축산물`은 하루 중 언제 가장 많이 팔리는가?

In [46]:
tr.query('상품중분류명 == "축산물"').구매시간.value_counts().index[0]

18

In [166]:
#농&축산물이 가장 많이 팔리는 시간 (예시}
tr.query('상품중분류명 in ["축산물", "농산물"]').구매시간.value_counts().index[0]

17

**[연습문제 40]** 전 지역에서 `판매건수가 가장 많은 상품`의 `총 매출액`은 얼마인가? Hint: 난이도 매우 높음.

In [167]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0


In [174]:
k = tr.상품중분류명.value_counts().index[0]
k

'가공식품'

In [191]:
#상품중분류명으로 그룹바이해서 총매출액을 구함.
tr.query('상품중분류명 == @k')['구매금액'].sum()

218379000

In [48]:
p = tr.상품중분류명.value_counts().index[0]

In [49]:
tr.query('상품중분류명 == @p').구매금액.sum()
# @사용.

218379000

**[연습문제 40]** 전 지역에서 `판매량이 가장 많은 상품`의 `총 매출액`은 얼마인가? Hint: 난이도 매우 높음.

In [50]:
tr.groupby('상품중분류명')['구매수량'].sum().sort_values(ascending = False).index[0]
#groupby ~로 묶어라 라는 의미를 가지고 있음.

#자동으로 내림차순 정렬을 하는 것은 오직 시리즈 타입에 사용할 수 있는 value_counts()임 
#---> sum을 내림차순 정렬하기 위해서는 sort_values()를 사용

'가공식품'

<font color = "#CC3D3D"><p>
# End